# Visualisation 

In [122]:
import numpy as np
import cv2 as cv2

In [123]:
def show(img, factor=1,name="image"):
    """ 
    show an image until the escape key is pressed
    :param factor: scale factor (default 1, half size)
    """
    if factor != 1.0:
        img = cv2.resize(img, (0,0), fx=factor, fy=factor) 

    cv2.imshow(name,img)
    while(1):
        k = cv2.waitKey(0)
        if k==27:    # Esc key to stop
            break
    cv2.destroyAllWindows()

In [147]:
def read_image(input_path):
    img = cv2.imread(input_path,0)
    return img

## Dividing Paper

In [220]:
def remove_top(img):
    canny = cv2.Canny(img, 100, 255, 1)
    show(canny,0.2)
    a = np.sum(canny, axis=1)
    print(np.argsort(np.argmax(a , axis = 0 )))
    return img

# Main

In [221]:
file_path = "F://Tech/CUFE_CHS/Fall_2020/Pattern/Project/Data_Set/a01-000u.png"
img = read_image(file_path)
show(img,0.2)
img = remove_top(img)
show(img, 0.2)

[0]
